In [3]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("someProblems").getOrCreate()

25/11/28 20:06:05 WARN Utils: Your hostname, user-HP-Pavilion-x360-Convertible-14-dh0xxx resolves to a loopback address: 127.0.1.1; using 192.168.1.24 instead (on interface wlo1)
25/11/28 20:06:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/28 20:06:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data = [("a", "aa", 1),("a", "aa", 2),("b", "bb", 5),("b", "bb", 3),("b", "bb", 4)] 
schema = ["col1", "col2", "col3"] 

df = spark.createDataFrame(data, schema)
df.show()
df.printSchema()

In [9]:
from pyspark.sql.functions import collect_list
df2 = df.groupBy("col1","col2").agg(collect_list("col3").alias("col3"))
df2.show()

+----+----+---------+
|col1|col2|     col3|
+----+----+---------+
|   a|  aa|   [1, 2]|
|   b|  bb|[5, 3, 4]|
+----+----+---------+



In [10]:
df.createOrReplaceTempView("myView") 

In [11]:
df3 = spark.sql("select col1, col2, collect_list(col3) as col3 from myView group by col1, col2")  
df3.show()

+----+----+---------+
|col1|col2|     col3|
+----+----+---------+
|   a|  aa|   [1, 2]|
|   b|  bb|[5, 3, 4]|
+----+----+---------+



In [17]:
from pyspark.sql.types import StructType, StructField, IntegerType, ArrayType
schema = StructType([
         StructField("dept_id", IntegerType(), True),
         StructField("e_id", ArrayType(IntegerType()), True) 
         ])

In [13]:
df_j = spark.read.json("data.json") 

In [14]:
df_j.printSchema()

root
 |-- dept_id: long (nullable = true)
 |-- e_id: array (nullable = true)
 |    |-- element: long (containsNull = true)



In [20]:
from pyspark.sql.functions import explode
df_explode = df_j.select("dept_id", explode("e_id").alias("e_id")) 
df_explode.show()

+-------+-----+
|dept_id| e_id|
+-------+-----+
|    101|10101|
|    101|10102|
|    101|10103|
|    102|10201|
|    102|10202|
+-------+-----+

